## Imports

In [3]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 460.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 678.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 1.7 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 KB 10.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━

In [1]:
import os
import sys
from torch import nn
import random
import torch
import platform
from typing import Callable, List, Optional, Dict
import numpy as np
import scipy.sparse as sp

print(torch.__version__)
print(torch.version.cuda)
print(platform.python_version())

# from torch_geometric.data import (
# #     HeteroData,
#     Data,
#     InMemoryDataset,
#     Batch
#     )
# from torch_geometric.data.storage import EdgeStorage
# import torch_geometric.datasets as datasets
# import torch_geometric.transforms as transforms
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

seed = 1234
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

ModuleNotFoundError: No module named 'scipy'

## Dataset creation

### Graph file locations

In [2]:
folders = ["/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/After pruning/new/PreparedStatement.executeQuery/", 
           "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/After pruning/new/Calendar.getTime/",
           "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/After pruning/new/URL.openConnection/",
           "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/After pruning/new/BufferedReader.readLine/"]

### Method to process raw graph data(in .txt format)

In [3]:
def get_nodes_edges(inTextFile, add_reverse_edges = False):
  # FD = 0, CD = 1
  # to support the hetero data object as suggested by the documentation 
  nodes_dict = {}
  edge_indices_CD = []
  edge_indices_FD = []

  #to support the Data object as used by the Entities dat object as used in RGAT source code
  edge_indices = []
  edge_type = []
  
  # nodes_dict is an index_map
  node_count=0
  with open(inTextFile) as fp:
    
    file_name = inTextFile.split("/")[-1].strip()

    Lines = fp.readlines()
    for line in Lines:

      N = line.split('-->')
      N[0], N[1] = N[0].strip(), N[1].strip()
      
      #t1 = N[0].split('$$')   
      src = N[0].strip()   
      if src not in nodes_dict.keys():
        nodes_dict[src] = node_count
        node_count+=1
        
      #t2 = N[1].split('$$')
      right_idx = N[1].rfind('[')
      dst = N[1][:right_idx].strip()
      if dst not in nodes_dict.keys():
        nodes_dict[dst] = node_count
        node_count+=1

      x = N[1].strip()[right_idx + 1 : -1].strip()
      if(x == 'FD'):
        y=0
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
        edge_indices_FD.append([nodes_dict[src], nodes_dict[dst]])
      else: 
        y=1
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
        edge_indices_CD.append([nodes_dict[src], nodes_dict[dst]])
     
  return nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type, file_name

In [4]:
file = "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/After pruning/new/BufferedReader.readLine/0_sample-1_BufferedReader.readLine_graph_dump.txt"

get_nodes_edges(file, add_reverse_edges = False)

PermissionError: [Errno 13] Permission denied: '/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/After pruning/new/BufferedReader.readLine/0_sample-1_BufferedReader.readLine_graph_dump.txt'

### Method to get CodeBERT embedding for nodes

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

#os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4"
#device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = AutoModel.from_pretrained("microsoft/codebert-base")
#model.to(device)

In [ ]:
def get_node_embedding_from_codebert(nodes):
    list_of_embeddings = []
    for code_line in nodes.keys():
        code_line = code_line.split("$$")[1].strip()
        code_tokens = tokenizer.tokenize(code_line)
        tokens = [tokenizer.cls_token]+code_tokens+[tokenizer.eos_token]
        tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
        context_embeddings = codebert_model(torch.tensor(tokens_ids)[None,:])
        cls_token_embedding = context_embeddings.last_hidden_state[0,0,:]
        list_of_embeddings.append(cls_token_embedding)
    return torch.stack(list_of_embeddings)

### Method to create data objects for GNN

In [ ]:
import glob
import tqdm
import sys

def create_graph_dataset(folders):
  dataset =[]
  for label, folder in tqdm.tqdm(enumerate(folders)):
    print("\nProcessing: {}\n".format(folder))
    files = glob.glob(os.path.join(folder, '*.txt'))
    print("\nNumber of files: {}\n".format(len(files)))
    count = 0
    for file in files:

      if(count % 5 == 0):
        print("\nAt file: {}\n".format(count))
      nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type, file_name = get_nodes_edges(file, add_reverse_edges = True)
      if(len(nodes_dict) == 0):
        print("No Data: ", file)
        continue
      #print(nodes_dict, edge_indices_CD, edge_indices_FD, edge_type)

      # Node feature matrix with shape [num_nodes, num_node_features]=(N, 768).
      try:
        CodeEmbedding = get_node_embedding_from_codebert(nodes_dict)
      except Exception as e :
        print("Error: ", e)
        print(nodes_dict)
        sys.exit()
      #print(CodeEmbedding.shape)

      # FIXING DATA FOTMATS AND SHAPE
      x = torch.tensor(CodeEmbedding)
      # print(x.shape)
  
      # data.y: Target to train against (may have arbitrary shape),
      # graph-level targets of shape [1, *]
      y = torch.tensor([label], dtype=torch.long)
      #print(type(y))

      # edge_index (LongTensor, optional) – Graph connectivity in COO format with shape [2, num_edges]
      edge_index_CD = torch.tensor(edge_indices_CD, dtype=torch.long).t().contiguous()
      edge_index_FD = torch.tensor(edge_indices_FD, dtype=torch.long).t().contiguous()
      edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
      edge_type = torch.tensor(edge_type, dtype=torch.long).t().contiguous()
      #print(edge_index_CD, edge_index_FD, edge_index, edge_type)
  
      data = Data(edge_index=edge_index, edge_type=edge_type, x=x)
      data.y = y
      data.num_nodes = len(nodes_dict)
      data.api = file_name
      dataset.append(data)
      count += 1
    
  return dataset

dataset = create_graph_dataset(folders)
print("\nLength of the dataset: ", len(dataset))

### Save the dataset

In [ ]:
for data in dataset:
    torch.save(data, f'/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/Graph data/no_reverse_edge/{data.api}.pt')

### Load data from existing file 

In [ ]:
import glob
import torch 

dataset_folder_path = "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Processed Dataset/Graph data/new"
graph_data_files = glob.glob(os.path.join(dataset_folder_path, '*.pt'))
dataset = []
for data_file in graph_data_files:
    data = torch.load(data_file)
    dataset.append(data)

print("\nLength of the dataset: ", len(dataset))

### Filter the datapoints if needed

In [ ]:
print("\nLength of the dataset: ", len(dataset))
filtered_dataset = []
for data in dataset:
    if data.num_nodes >=3 :
        filtered_dataset.append(data)
        
print("\nLength of the filtered dataset: ", len(filtered_dataset))

### Graph Statistics & Visualization

In [ ]:
print("\nLength of the dataset: ", len(filtered_dataset))

data = filtered_dataset[0]
print(data, "\n")
print('=============================================================')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')
print(data.edge_index)
print(data.edge_type)
#print(data.x.dtype, data.edge_index.dtype, data.edge_type.dtype, data.y.dtype)

In [ ]:
# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

def visualize_graph(data, color):
    G = to_networkx(data, to_undirected=False)
    edge_labels = {}
    for i in range(len(G.edges)):
        edge = list(G.edges)[i]
        edge_labels[edge[0], edge[1]] = int(data.edge_type[i])
    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=True,
                     node_color=color, cmap="Set2")
    nx.draw_networkx_edge_labels(G, pos=nx.spring_layout(G, seed=42), edge_labels=edge_labels, font_color='red')
    plt.show()

for G in filtered_dataset[30:35]:
    visualize_graph(G, color="blue")

### Create train and test splits

In [ ]:
Label_counter = {"PreparedStatement.executeQuery" : 0, "Calendar.getTime": 0, "URL.openConnection": 0, "BufferedReader.readLine": 0}
for data in filtered_dataset:
    for label in Label_counter:
        if label in data.api:
            Label_counter[label] += 1
            break
print("API labels: ", Label_counter)

In [ ]:
shuffled_dataset = random.sample(filtered_dataset, len(filtered_dataset))

train_dataset = shuffled_dataset[:int(len(filtered_dataset) * 0.95)]
test_dataset = shuffled_dataset[int(len(filtered_dataset) * 0.95):]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

### Create the dataloaders

In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# for step, data in enumerate(train_loader):
#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()

## Model

### Define the GCN model

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features = 768, num_classes = 4):
        super(GCN, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)

        # 2. Readout layer
        graph_embedding = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(graph_embedding, p=0.5, training=self.training)
        x = self.lin(x)
        return x, graph_embedding

model = GCN(hidden_channels=64)
print(model)

### Define the GAT model

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features = 768, num_classes = 4):
        super(GAT, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GATConv(num_node_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)

        # 2. Readout layer
        graph_embedding = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(graph_embedding, p=0.5, training=self.training)
        x = self.lin(x)
        return x, graph_embedding

model = GAT(hidden_channels=64)
print(model)

### Visualizing the embeddings

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap

def visualize_embeddings(h, color):
    z = umap.UMAP(n_components=2).fit_transform(h.detach().cpu().numpy())
    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [ ]:
model = GCN(hidden_channels=64)

model.eval()
dataloader = DataLoader(shuffled_dataset, len(shuffled_dataset), shuffle=False)
for data in dataloader:
    out, graph_embedding = model(data.x, data.edge_index, data.batch)
    visualize_embeddings(graph_embedding, color=data.y)

### Training & Testing

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6,10"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, torch.cuda.device_count(), torch.cuda.current_device())

In [ ]:
model = GAT(hidden_channels=64)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:  # Iterate in batches over the training dataset.
         data.to(device)
         out, _ = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         #print("Out = {} and Label = {}".format(out, data.y))
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.
         total_loss += float(loss)
    print("Loss: ", total_loss)

def test(loader):
     model.eval()
     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         data.to(device)
         out, _ = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(0, 5):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
model.to(torch.device("cpu"))
for data in DataLoader(shuffled_dataset, len(shuffled_dataset), shuffle=False):
    out, graph_embedding = model(data.x, data.edge_index, data.batch)
    visualize_embeddings(graph_embedding, color=data.y)

### Save Model

In [ ]:
PATH = "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Saved_models/Graph_classification/GAT/model_with_reverse_edge.pt"
torch.save(model.state_dict(), PATH)

### Load Model

In [ ]:
PATH = "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/Saved_models/Graph_classification/model_2.pt"
model = GCN(hidden_channels=64)
model.load_state_dict(torch.load(PATH))
model.eval()

## Cluster the embeddings

### Collect embeddings for a specific class or API

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

CLASS_LABEL = 2

model.eval()
train_api_embeddings, train_api_datapoints = [], []
test_api_embeddings, test_api_datapoints = [], []

for data in DataLoader(test_dataset, len(test_dataset), shuffle=False):
    _, graph_embedding = model(data.x, data.edge_index, data.batch)
    for i in range(len(test_dataset)):
        if(int(data[i].y) == CLASS_LABEL):
            test_api_datapoints.append(data[i])
            test_api_embeddings.append(graph_embedding[i].detach().cpu().numpy())
print(len(test_api_embeddings), len(test_api_datapoints))

for data in DataLoader(train_dataset, len(train_dataset), shuffle=False):
    _, graph_embedding = model(data.x, data.edge_index, data.batch)
    for i in range(len(train_dataset)):
        if(int(data[i].y) == CLASS_LABEL):
            train_api_datapoints.append(data[i])
            train_api_embeddings.append(graph_embedding[i].detach().cpu().numpy())
print(len(train_api_embeddings), len(train_api_datapoints))

total_api_embeddings, total_api_datapoints = train_api_embeddings + test_api_embeddings, train_api_datapoints + test_api_datapoints
print(len(total_api_embeddings), len(total_api_datapoints))

### K-Means

#### Get optimal clusters number

In [ ]:
def elbow_for_kmeans(embeddings):

    inertias, no_of_clusters = [], list(range(1, len(embeddings)))

    for i in no_of_clusters:
        kmeans = KMeans(n_clusters=i)
        kmeans.fit(embeddings)
        inertias.append(kmeans.inertia_)

    plt.plot(no_of_clusters, inertias, marker='o')
    plt.title('Elbow method')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.show()
    
elbow_for_kmeans(train_api_embeddings)

#### Do clustering

In [ ]:
kmeans = KMeans(n_clusters = 10)
kmeans.fit(train_api_embeddings)
test_api_clusters_kmeans = kmeans.predict(test_api_embeddings)
print(test_api_clusters_kmeans)

#### Visualize using t-SNE 

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
import pandas as pd

def visualize_clusters_using_tsne(vectors, clusters):

    tsne_output_2d = TSNE(n_components = 2).fit_transform(vectors)
    tsne_dataframe_2d = pd.DataFrame(tsne_output_2d)
    tsne_dataframe_2d['cluster'] = clusters
    tsne_dataframe_2d.columns = ['x1','x2','cluster']

    tsne_output_3d = TSNE(n_components = 3).fit_transform(vectors)
    tsne_dataframe_3d = pd.DataFrame(tsne_output_3d)
    tsne_dataframe_3d['cluster'] = clusters
    tsne_dataframe_3d.columns = ['x1','x2', 'x3', 'cluster']

    fig = px.scatter(
        tsne_dataframe_2d[["x1", "x2"]], x="x1", y="x2",
        color=tsne_dataframe_2d["cluster"], labels={'color': 'clusters'}
    )
    fig.show()

    fig = px.scatter_3d(
        tsne_dataframe_3d[["x1", "x2", "x3"]], x="x1", y="x2", z="x3",
        color=tsne_dataframe_3d["cluster"], labels={'color': 'clusters'}
    )
    fig.update_traces(marker_size = 8)
    fig.show()

visualize_clusters_using_tsne(train_api_embeddings, kmeans.predict(train_api_embeddings))

#### Visualize using UMAP

In [ ]:
from umap import UMAP
import plotly.express as px

def visualize_clusters_using_umap(vectors, clusters):

    umap_2d = UMAP(n_components=2, init='random', random_state=0)
    umap_output_2d = umap_2d.fit_transform(vectors)
    umap_dataframe_2d = pd.DataFrame(umap_output_2d)
    umap_dataframe_2d['cluster'] = clusters
    umap_dataframe_2d.columns = ['x1','x2', 'cluster']

    umap_3d = UMAP(n_components=3, init='random', random_state=0)
    umap_output_3d = umap_3d.fit_transform(vectors)
    umap_dataframe_3d = pd.DataFrame(umap_output_3d)
    umap_dataframe_3d['cluster'] = clusters
    umap_dataframe_3d.columns = ['x1','x2', 'x3', 'cluster']

    fig_2d = px.scatter(
        umap_dataframe_2d[["x1", "x2"]], x="x1", y="x2",
        color=umap_dataframe_2d["cluster"], labels={'color': 'clusters'}
    )
    fig_3d = px.scatter_3d(
        umap_dataframe_3d[["x1", "x2", "x3"]], x="x1", y="x2", z="x3",
        color=umap_dataframe_3d["cluster"], labels={'color': 'clusters'}
    )
    fig_3d.update_traces(marker_size=5)

    fig_2d.show()
    fig_3d.show()
    
visualize_clusters_using_umap(train_api_embeddings, kmeans.predict(train_api_embeddings))

### BIRCH Clustering

In [ ]:
from sklearn.cluster import Birch

model.eval()
filtered_dataset_api_embeddings, filtered_dataset_api_datapoints = [], []

for data in DataLoader(filtered_dataset, len(filtered_dataset), shuffle=False):
    _, graph_embedding = model(data.x, data.edge_index, data.batch)
    for i in range(len(filtered_dataset)):
        filtered_dataset_api_datapoints.append(data[i])
        filtered_dataset_api_embeddings.append(graph_embedding[i].detach().cpu().numpy())
print(len(filtered_dataset_api_embeddings), len(filtered_dataset_api_datapoints))

birch_model = Birch(n_clusters = None)
filtered_dataset_clusters_birch = birch_model.fit_predict(train_api_embeddings)
print(len(set(filtered_dataset_clusters_birch)))

#### Do clustering

In [ ]:
from sklearn.cluster import Birch

birch_model = Birch(n_clusters = None)
train_api_clusters_birch = birch_model.fit_predict(train_api_embeddings)
test_api_clusters_birch = birch_model.predict(test_api_embeddings)


In [ ]:
d = {}
for i in train_api_clusters_birch:
    if i in d:
        d[i] += 1
    else:
        d[i] = 1
print(d)

#### Visualize using t-SNE

In [ ]:
visualize_clusters_using_tsne(train_api_embeddings, train_api_clusters_birch)

#### Visualize using UMAP

In [ ]:
visualize_clusters_using_umap(train_api_embeddings, train_api_clusters_birch)

### DBSCAN Clustering

#### Do clustering

In [ ]:
from sklearn.cluster import DBSCAN

train_api_clusters_dbscan = DBSCAN(eps=10, min_samples=4).fit_predict(train_api_embeddings)
print("Clusters: {}".format(train_api_clusters_dbscan))

#### Visualize using TSNE

In [ ]:
visualize_clusters_using_tsne(train_api_embeddings, train_api_clusters_dbscan)

#### Visualize using UMAP

In [ ]:
visualize_clusters_using_umap(train_api_embeddings, train_api_clusters_dbscan)

## Compare with Code-Kernel

### Get Code-Kernel cluster details for a particular API

In [ ]:
print("\nAPI name: ", total_api_datapoints[0].api)

code_kernel_clusters = {}
code_kernel_api_cluster_mapping = {}

for data in total_api_datapoints:
    name_parts = data.api.split("_")
    file_name, cluster_no, api_name = name_parts[0].strip(), name_parts[1].strip(), name_parts[2].strip()
    if cluster_no in code_kernel_clusters:
        code_kernel_clusters[cluster_no].append(data.api)
    else:
        code_kernel_clusters[cluster_no] = [data.api]
    code_kernel_api_cluster_mapping[data.api] = cluster_no
        
print("\nTotal data points: ", len(code_kernel_api_cluster_mapping))
print("\nNumber of cluster in Code-Kernel: ", len(code_kernel_clusters))
for cluster_no in code_kernel_clusters:
    print("{} : {}".format(cluster_no, len(code_kernel_clusters[cluster_no])))

### Create cluster-classification ground-truth from Code-Kernel data

In [ ]:
def do_classification_from_clustering(data_to_cluster_map):
    classification = {}
    data_points = list(data_to_cluster_map.keys())
    for i in range(len(data_points)):
        j = i+1
        while j < len(data_points):
            classification[tuple([data_points[i], data_points[j]])] = 1 if data_to_cluster_map[data_points[i]] == data_to_cluster_map[data_points[j]] else 0
            j += 1
    return classification

In [ ]:
code_kernel_classification = do_classification_from_clustering(code_kernel_api_cluster_mapping)
print("\nTotal code-kernel classifications : ", len(code_kernel_classification))

### Cluster GNN embeddings with same cluster numbers

In [ ]:
from sklearn.cluster import Birch

birch_model = Birch(n_clusters = 17)
total_api_clusters_birch = birch_model.fit_predict(total_api_embeddings)

total_api_cluster_mapping = {}
for data, cluster in zip(total_api_datapoints, total_api_clusters_birch):
    total_api_cluster_mapping[data.api] = cluster
    
print("\nTotal data points: ", len(total_api_cluster_mapping))
total_api_classification = do_classification_from_clustering(total_api_cluster_mapping)
print("\nTotal GNN classifications : ", len(total_api_classification))

### Compare GNN clustering accuracy wrt Code-Kernel

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

code_kernel_labels, GNN_labels = [], []
for pair in total_api_classification:
    code_kernel_labels.append(code_kernel_classification[pair])
    GNN_labels.append(total_api_classification[pair])
    
print("\nConfusion Matrix: ", confusion_matrix(code_kernel_labels, GNN_labels))
print("\nAccuracy: ", accuracy_score(code_kernel_labels, GNN_labels) * 100)
print("\nF1-Score: ", f1_score(code_kernel_labels, GNN_labels) * 100)
print("\nPrecision: ", precision_score(code_kernel_labels, GNN_labels) * 100)
print("\nRecall: ", recall_score(code_kernel_labels, GNN_labels) * 100)

### Get some wrongly classified datapoints for analysis

## Compare with Code-Kernel using manually labelled data

### Create the graph dataset

In [ ]:
import glob
import tqdm
import sys

folder_path = "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/CodeKernel_Manual_Data/Processed_data/after_pruning/new"
pdg_folders_list = glob.glob(folder_path + "/*/")
print("\nNumber of total APIs: {}\n".format(len(pdg_folders_list)))

manual_dataset = create_graph_dataset(pdg_folders_list)
print("\nLength of the manually labelled dataset: ", len(manual_dataset))

In [ ]:
for data in manual_dataset:
    torch.save(data, f'/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/CodeKernel_Manual_Data/Processed_data/graph_data/no_reverse_edge/{data.api}.pt')

In [ ]:
print("\nLength of the manual dataset: ", len(manual_dataset))
filtered_manual_dataset = []
for data in manual_dataset:
    if data.num_nodes >= 3 :
        filtered_manual_dataset.append(data)
        
print("\nLength of the filtered manual dataset: ", len(filtered_manual_dataset))

In [ ]:
print("\nLength of the dataset: ", len(filtered_manual_dataset))

data = filtered_manual_dataset[0]
print(data, "\n")
print('=============================================================')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')
print(data.edge_index)
print(data.edge_type)

### Load existing data

In [ ]:
import glob
import torch 

dataset_folder_path = "/raid/cs21mtech12001/API-Misuse-Research/PDG-Gen/Repository/CodeKernel_Manual_Data/Processed_data/graph_data/new/"
graph_data_files = glob.glob(os.path.join(dataset_folder_path, '*.pt'))
manual_dataset = []
for data_file in graph_data_files:
    data = torch.load(data_file)
    manual_dataset.append(data)

print("\nLength of the manual dataset: ", len(manual_dataset))
filtered_manual_dataset = []
for data in manual_dataset:
    if data.num_nodes >= 3 :
        filtered_manual_dataset.append(data)
        
print("\nLength of the filtered manual dataset: ", len(filtered_manual_dataset))

### Get the embeddings

In [ ]:
model.eval()
manual_data_embeddings = {}

for data in DataLoader(manual_dataset, len(manual_dataset), shuffle=False):
    _, graph_embedding = model(data.x, data.edge_index, data.batch)
    for i in range(len(manual_dataset)):
        manual_data_embeddings[data[i].api] = graph_embedding[i].detach().cpu().numpy()
    
print(len(manual_data_embeddings))

### Get cluster labels for Ground-truth, Code-Kernel and GNN

In [ ]:
import copy

APIs = ["Driver.connect", "FileUtils.writeStringToFile", "Graphics2D.fill", "IOUtils.toString", "PrinterJob.pageDialog", "Properties.loadFromXML", 
        "Servant._poa", "Window.pack"]

gt_labels_for_manual_data = dict.fromkeys(APIs, 0)
ck_labels_for_manual_data = dict.fromkeys(APIs, 0)
gnn_labels_for_manual_data = dict.fromkeys(APIs, 0)

for api in APIs:
    gt_labels_for_manual_data[api] = copy.deepcopy(dict({}))
    ck_labels_for_manual_data[api] = copy.deepcopy(dict({}))
    gnn_labels_for_manual_data[api] = copy.deepcopy(dict({}))

for data in manual_dataset:
    #print(data.api)
    parts = data.api.strip().split("_")
    ck_label, gt_label = -1, -1
    flag = False
    for i in range(len(parts)):
        if "sample" in parts[i]:
            gt_label = int(parts[i].split("-")[1])
            try:
                ck_label = int(parts[i-1])
            except ValueError:
                print("\nValue error for : ", data.api)
                ck_label = 1
            flag = True
            break
    if not flag:
        print("\nck/gt label not found")
    flag = False
    for api in APIs:
        if api in data.api:
            #print(api, data.api)
            gt_labels_for_manual_data[api][data.api] = gt_label
            ck_labels_for_manual_data[api][data.api] = ck_label
            gnn_labels_for_manual_data[api][data.api] = -1
            flag = True            
            break
    if not flag:
        print("\nAPI label is not set")

In [ ]:
total_points_gt, total_points_ck, total_points_gnn = 0, 0, 0

for api in gt_labels_for_manual_data:
 total_points_gt += len(gt_labels_for_manual_data[api])
 total_points_ck += len(ck_labels_for_manual_data[api])
 total_points_gnn += len(gnn_labels_for_manual_data[api])
 
print(total_points_gt, total_points_ck, total_points_gnn)

### Cluster using BIRCH clustering

In [ ]:
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')

for api in gnn_labels_for_manual_data.keys():
    embeddings_as_list, file_names = [], []
    for file_name in gnn_labels_for_manual_data[api].keys():
        embeddings_as_list.append(manual_data_embeddings[file_name])
        file_names.append(file_name)
        
    if len(embeddings_as_list) == 0:
        print("\nNo embeddings for api: ", api)
        continue
    
    try:
        birch_silhouette_scores= []
        no_of_clusters = list(range(2, len(embeddings_as_list)))
        for n_cluster in no_of_clusters:
            birch_silhouette_scores.append(silhouette_score(embeddings_as_list, Birch(n_clusters = n_cluster).fit_predict(embeddings_as_list)))
    
        correct_cluster_number, max_silhouette_scores = -1, -2
        for cluster_no, birch_silhouette_score in zip(no_of_clusters, birch_silhouette_scores):
            correct_cluster_number = cluster_no if max_silhouette_scores < birch_silhouette_score else correct_cluster_number
            max_silhouette_scores = birch_silhouette_score if max_silhouette_scores < birch_silhouette_score else max_silhouette_scores
        #print("\nMax Silhouette score for {} is {} for clusters {}".format(api, max_silhouette_scores, correct_cluster_number))
    except Exception as e:
        print("\nSilhouette error for API {} : {}".format(api, e))
        correct_cluster_number = None
    
    #correct_cluster_number = None
    birch_model = Birch(n_clusters = correct_cluster_number)
    clusters_birch = birch_model.fit_predict(embeddings_as_list)
    print("\nNumber of clusters for {} is {} having total {} datapoints".format(api, len(set(clusters_birch)), len(embeddings_as_list)))

    for file_name, cluster_no in zip(file_names, clusters_birch):
        gnn_labels_for_manual_data[api][file_name] = cluster_no

In [ ]:
for api in gnn_labels_for_manual_data.keys():
    for file_name in gnn_labels_for_manual_data[api].keys():
        if gnn_labels_for_manual_data[api][file_name] == -1:
            print("\nLabel not set for : ", file_name)

### Cluster using KMeans cluatering

In [ ]:
for api in gnn_labels_for_manual_data.keys():
    embeddings_as_list, file_names = [], []
    for file_name in gnn_labels_for_manual_data[api].keys():
        embeddings_as_list.append(manual_data_embeddings[file_name])
        file_names.append(file_name)
        
    print("\nFor ", api)
    
    elbow_for_kmeans(embeddings_as_list)

In [ ]:
KMeans_cluster_numbers = {"Driver.connect" : 5, "FileUtils.writeStringToFile" : 2, 
                          "Graphics2D.fill" : 5, "IOUtils.toString" : 2, 
                          "PrinterJob.pageDialog" : 8, "Properties.loadFromXML" : 6, 
                          "Servant._poa" : 4, "Window.pack" : 3}

for api in gnn_labels_for_manual_data.keys():
    embeddings_as_list, file_names = [], []
    for file_name in gnn_labels_for_manual_data[api].keys():
        embeddings_as_list.append(manual_data_embeddings[file_name])
        file_names.append(file_name)
        
    kmeans_model = KMeans(n_clusters = KMeans_cluster_numbers[api])
    clusters_kmeans = kmeans_model.fit_predict(embeddings_as_list)
    print("\nNumber of clusters for {} is {} having total {} datapoints".format(api, len(set(clusters_kmeans)), len(embeddings_as_list)))

    for file_name, cluster_no in zip(file_names, clusters_kmeans):
        gnn_labels_for_manual_data[api][file_name] = cluster_no

### Pair-wise classification labels

In [ ]:
gt_classification_labels, ck_classification_labels, gnn_classification_labels = {}, {}, {}

for api in APIs:
    gt_classification_labels[api] = do_classification_from_clustering(gt_labels_for_manual_data[api])
    ck_classification_labels[api] = do_classification_from_clustering(ck_labels_for_manual_data[api])
    gnn_classification_labels[api] = do_classification_from_clustering(gnn_labels_for_manual_data[api])
    
    print("\n", len(gt_labels_for_manual_data[api]), len(gt_classification_labels[api]), 
          len(ck_labels_for_manual_data[api]), len(ck_classification_labels[api]),
          len(gnn_labels_for_manual_data[api]), len(gnn_classification_labels[api]))


### Classification accuracy for Code-Kernel

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score,classification_report

for api in APIs:
    
    ck_classification_labels_as_list, gt_classification_labels_as_list = [], []
    
    for pair in gt_classification_labels[api]:
        gt_classification_labels_as_list.append(gt_classification_labels[api][pair])
        ck_classification_labels_as_list.append(ck_classification_labels[api][pair])
    
    Confusion_matrix = confusion_matrix(gt_classification_labels_as_list, ck_classification_labels_as_list)
    accuracy = "{:.2f}".format(accuracy_score(gt_classification_labels_as_list, ck_classification_labels_as_list) * 100)
    precision = "{:.2f}".format(precision_score(gt_classification_labels_as_list, ck_classification_labels_as_list) * 100)
    recall = "{:.2f}".format(recall_score(gt_classification_labels_as_list, ck_classification_labels_as_list) * 100)
    F1_score = "{:.2f}".format(f1_score(gt_classification_labels_as_list, ck_classification_labels_as_list) * 100)
    
    print("\nAccuracy for API: ", api)
    print("Accuracy: {}, precision: {}, recall: {} and f1-score: {}".format(accuracy, precision, recall, F1_score))
    # if api == "IOUtils.toString":
    #     print(gt_classification_labels_as_list, ck_classification_labels_as_list)

### Classification accuracy for GNN model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

for api in APIs:
    
    gt_classification_labels_as_list, gnn_classification_labels_as_list = [], []
    for pair in gt_classification_labels[api]:
        gt_classification_labels_as_list.append(gt_classification_labels[api][pair])
        gnn_classification_labels_as_list.append(gnn_classification_labels[api][pair])
        
    Confusion_matrix = confusion_matrix(gt_classification_labels_as_list, gnn_classification_labels_as_list)
    accuracy = "{:.2f}".format(accuracy_score(gt_classification_labels_as_list, gnn_classification_labels_as_list) * 100)
    precision = "{:.2f}".format(precision_score(gt_classification_labels_as_list, gnn_classification_labels_as_list) * 100)
    recall = "{:.2f}".format(recall_score(gt_classification_labels_as_list, gnn_classification_labels_as_list) * 100)
    F1_score = "{:.2f}".format(f1_score(gt_classification_labels_as_list, gnn_classification_labels_as_list) * 100)
    
    print("\nAccuracy for API: ", api)
    print("Accuracy: {}, precision: {}, recall: {} and f1-score: {}".format(accuracy, precision, recall, F1_score))
    
    if api == "FileUtils.writeStringToFile":
        print(gt_classification_labels_as_list, gnn_classification_labels_as_list)

In [ ]:
for api in gt_labels_for_manual_data.keys():
    if api != "Driver.connect":
        continue
    #print(gt_labels_for_manual_data[api])
    clusters = {}
    for api_example in gt_labels_for_manual_data[api].keys():
        if gt_labels_for_manual_data[api][api_example] in clusters:
            clusters[gt_labels_for_manual_data[api][api_example]].append(api_example)
        else:
            clusters[gt_labels_for_manual_data[api][api_example]] = [api_example]
    print(clusters)

In [ ]:
for api in gnn_labels_for_manual_data.keys():
    if api != "Driver.connect":
        continue
    #print(gt_labels_for_manual_data[api])
    clusters = {}
    for api_example in gnn_labels_for_manual_data[api].keys():
        if gnn_labels_for_manual_data[api][api_example] in clusters:
            clusters[gnn_labels_for_manual_data[api][api_example]].append(api_example)
        else:
            clusters[gnn_labels_for_manual_data[api][api_example]] = [api_example]
    print(clusters)

In [ ]:
for api in gnn_labels_for_manual_data.keys():
    if api != "Driver.connect":
        continue
    #print(gt_labels_for_manual_data[api])
    clusters = {}
    for api_example in gnn_labels_for_manual_data[api].keys():
        if "sample-2_" in api_example:
            if gnn_labels_for_manual_data[api][api_example] in clusters:
                clusters[gnn_labels_for_manual_data[api][api_example]].append(api_example)
            else:
                clusters[gnn_labels_for_manual_data[api][api_example]] = [api_example]
    print(clusters)

In [ ]:
api_examples = ['M_connect_131_ConnectionManager_0_0_sample-0_Driver.connect_graph_dump.txt', 'M_connect_132_ConnectionManager_0_0_sample-0_Driver.connect_graph_dump.txt', 'M_connect_114_SQLGet_0_0_sample-0_Driver.connect_graph_dump.txt']
for data in manual_dataset:
    if data.api in api_examples:
        visualize_graph(data, color="blue")